<a href="https://colab.research.google.com/github/tokyo8182/LLM-RAG/blob/main/Embeddings_Inference_Time_Comparison_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index
%pip install "llama-index-embeddings-huggingface"
%pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 56.2 MB/s eta 0:00:00


## Setup + Data

In [2]:
import pandas as pd
import time
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

In [5]:

# connecting to google drive drive:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def load_data(csv_path):
    return pd.read_csv(csv_path)['markdown_content'].tolist()

DATA_PATH = "/content/drive/MyDrive/Omdena_Challenge/new_LK_tea_dataset/v2_new_LK_tea_dataset_updated.csv"
documents = load_data(DATA_PATH)
len(documents)

166

In [7]:
# drop nans
documents = [doc for doc in documents if isinstance(doc, str) and not pd.isna(doc)]
len(documents)

165

In [8]:
words_per_doc = [len(str(document).split(' ')) for document in documents]
len(words_per_doc), sum(words_per_doc), max(words_per_doc), min(words_per_doc)

(165, 167135, 21832, 99)

## Chunking

In [9]:
def generate_chunk_nodes(texts):
    # Text chunking
    splitter = SentenceSplitter(chunk_size=750, chunk_overlap=250)
    nodes = splitter.get_nodes_from_documents(
        [Document(text=text) for text in texts]
    )
    return nodes

chunk_nodes = generate_chunk_nodes(documents)

print(f"Total chunks created: {len(chunk_nodes)}")

Total chunks created: 575


In [10]:
def measure_embedding_time(model_name, chunk_nodes):
    # Initialize embedding model
    embed_model = HuggingFaceEmbedding(model_name=model_name, trust_remote_code=True, cache_folder="/content/drive/MyDrive/Omdena_Challenge/cached_models/")

    # Time embedding process
    start_time = time.monotonic()
    embeddings = [embed_model.get_text_embedding(node.text) for node in chunk_nodes]

    end_time = time.monotonic()
    return end_time - start_time, len(embeddings), embeddings

## Embeddings Generation

### [bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5)

In [11]:
model_name = 'BAAI/bge-small-en-v1.5'
elapsed_time_bge, embd_count_bge, bge_embeddings_bge = measure_embedding_time(model_name, chunk_nodes)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
print(f"Generated {embd_count_bge} embeddings in {elapsed_time_bge:.4f} seconds")
print(f"Generation rate {(embd_count_bge/elapsed_time_bge):.4f} embeddings/second")

Generated 575 embeddings in 12.0393 seconds
Generation rate 47.7604 embeddings/second


## [stella_en_400M_v5](https://huggingface.co/dunzhang/stella_en_400M_v5)

In [13]:
model_name = 'dunzhang/stella_en_400M_v5'
elapsed_time_st, embd_count_st, nv_st_embeddings = measure_embedding_time(model_name, chunk_nodes)

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/170k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

In [14]:
print(f"Generated {embd_count_st} embeddings in {elapsed_time_st:.4f} seconds")
print(f"Generation rate {(embd_count_st/elapsed_time_st):.4f} embeddings/second")

Generated 575 embeddings in 74.7813 seconds
Generation rate 7.6891 embeddings/second
